In [ ]:
# Required connection configs for Kafka producer, consumer, and admin
bootstrap.servers=pkc-921jm.us-east-2.aws.confluent.cloud:9092
security.protocol=SASL_SSL
sasl.mechanisms=PLAIN
sasl.username=RKMBW5IUDCVGJHRB
sasl.password=cfltBPTHcyr9cJ9qtju+MT3B9AUyIiSV7ERmB8/tpoxx+MhU9aqu47ogO0aeDPsA

# Best practice for higher availability in librdkafka clients prior to 1.7
session.timeout.ms=45000

client.id=ccloud-python-client-0ba89b25-4c4f-437d-b6a2-47efab204258


In [2]:
import pandas as pd
import json

csv_file="first_100_customers.csv"
df=pd.read_csv(csv_file)
df.head()

,customer_id,name,city,state,country,registration_date,is_active
0,0,Customer_0,Pune,Maharashtra,India,2023-06-29,False
1,1,Customer_1,Bangalore,Tamil Nadu,India,2023-12-07,True
2,2,Customer_2,Hyderabad,Gujarat,India,2023-10-27,True
3,3,Customer_3,Bangalore,Karnataka,India,2023-10-17,False
4,4,Customer_4,Ahmedabad,Karnataka,India,2023-03-14,False


In [3]:
json_records=df.to_dict(orient="records")

json_file ="customers.json"

with open(json_file,'w') as file:
    json.dump(json_records,file,indent=4)
    
print("File Converted to JSON")

File Converted to JSON


In [1]:
!pip install confluent-Kafka

  Obtaining dependency information for confluent-Kafka from https://files.pythonhosted.org/packages/5e/a0/30a2b36b359d36f4b71cc49522fe5a3ef96a9a59a7df1f3a86f6c758d01b/confluent_kafka-2.11.1-cp311-cp311-win_amd64.whl.metadata
   ---------------------------------------- 0.0/4.0 MB ? eta -:--:--
   ---------- ----------------------------- 1.0/4.0 MB 33.4 MB/s eta 0:00:01
   -------------------- ------------------- 2.1/4.0 MB 26.7 MB/s eta 0:00:01
   -------------------------------------- - 3.9/4.0 MB 27.9 MB/s eta 0:00:01
   ---------------------------------------- 4.0/4.0 MB 25.8 MB/s eta 0:00:00


In [4]:
from confluent_kafka import Producer
import json
import time

conf = {
    "bootstrap.servers": "pkc-921jm.us-east-2.aws.confluent.cloud:9092",
    "security.protocol": "SASL_SSL",
    "sasl.mechanisms": "PLAIN",
    "sasl.username": "RKMBW5IUDCVGJHRB",
    "sasl.password": "cfltBPTHcyr9cJ9qtju+MT3B9AUyIiSV7ERmB8/tpoxx+MhU9aqu47ogO0aeDPsA",
    "session.timeout.ms": 45000,
    "client.id": "ccloud-python-client-0ba89b25-4c4f-437d-b6a2-47efab204258"
}

producer = Producer(conf)


In [7]:
topic ='ecommerce'

with open('customers.json','r') as file:
    customers_data=json.load(file)
    
value=customers_data[0]
key=value['customer_id']

print(key,value)

0 {'customer_id': 0, 'name': 'Customer_0', 'city': 'Pune', 'state': 'Maharashtra', 'country': 'India', 'registration_date': '2023-06-29', 'is_active': False}


In [8]:
producer.produce(topic,key=str(key).encode('utf-8'),value=str(value).encode('utf-8'))

In [9]:
#sending multiple messages

In [13]:
topic = 'ecommerce'



def delivery_status(err,msg):
    if(err):
        print(f"Message delivery failed: {err}")
    else:
        print(f"Message delivered to {msg.topic()} [{msg.partition()}] at offset {msg.offset()}")

for record in customers_data:
    try:
        message_value = json.dumps(record)
        message_key = str(record['customer_id']).encode('utf-8')

        producer.produce(topic,key = message_key,value=message_value,callback = delivery_status)
        producer.poll(1)

    except Exception as e:
        print(f"Error sending messages: {e}")

producer.flush()

print("Message send to kafka successfully")


Message delivered to ecommerce [0] at offset 133
Message delivered to ecommerce [2] at offset 75
Message delivered to ecommerce [0] at offset 134
Message delivered to ecommerce [1] at offset 94
Message delivered to ecommerce [0] at offset 135
Message delivered to ecommerce [2] at offset 76
Message delivered to ecommerce [0] at offset 136
Message delivered to ecommerce [1] at offset 95
Message delivered to ecommerce [0] at offset 137
Message delivered to ecommerce [1] at offset 96
Message delivered to ecommerce [2] at offset 77
Message delivered to ecommerce [1] at offset 97
Message delivered to ecommerce [2] at offset 78
Message delivered to ecommerce [0] at offset 138
Message delivered to ecommerce [1] at offset 98
Message delivered to ecommerce [0] at offset 139
Message delivered to ecommerce [0] at offset 140
Message delivered to ecommerce [2] at offset 79
Message delivered to ecommerce [1] at offset 99
Message delivered to ecommerce [1] at offset 100
Message delivered to ecommerce 